In [1]:
"""
Initialise the environment.
"""

import sys
from dotenv import load_dotenv
from pathlib import Path

load_dotenv()
if (parent_path := str(Path.cwd().parent)) not in sys.path:
    sys.path.append(parent_path)


def print_conversation(agent):
    for m in agent.conversation_history:
        print(m)


shop_query = "Where's the closest shop for Brillat de Savarine in Brooklyn?"
order_query = "Can you order some cranberry sauce?"

In [2]:
"""
Zero-shot agent: We let the agent choose the tool using only the name, inputs, and descriptions as guidance.
"""

from src.examples.zero_shot import init_agent as init_culinary_agent_zero_shot


async def zero_shot():
    agent = init_culinary_agent_zero_shot()
    await agent(shop_query)
    print_conversation(agent)


await zero_shot()

system: You are an experienced chef with a passion for creating delicious homemade meals.
You'll be working with your teammate to prepare a meal for Thanksgiving.

You have a set of tools at your disposal, which you may invoke when appropriate.
user: Where's the closest shop for Brillat de Savarine in Brooklyn?
assistant: [<ToolCall function=name='find_nearby_stores' arguments={'ingredient': 'Brillat de Savarine', 'location': 'Brooklyn'}>]
tool: {
  "name": "Healthy Harvest",
  "address": "789 Oak Ave",
  "distance": "0.8 miles"
}
assistant: The closest shop for Brillat de Savarine in Brooklyn is Healthy Harvest, located at 789 Oak Ave, just 0.8 miles away.


In [3]:
"""
Multi-shot agnostic agent: We provide examples of user inputs and assistant tool calls in our system prompt.
The examples are not represented as provider-specfiic tool invocations, but rather generic objects.
"""

from src.examples.multishot_agnostic import init_agent as init_multishot_agnostic_agent


async def multishot_agnostic():
    agent = init_multishot_agnostic_agent()
    await agent(shop_query)
    print_conversation(agent)


await multishot_agnostic()

system: You are an experienced chef with a passion for creating delicious homemade meals.
You'll be working with your teammate to prepare a meal for Thanksgiving.

You have a set of tools at your disposal, which you may invoke when appropriate.

Here are some examples to help you decide when to use which tool:
<examples>
<example>user: Hey, how are you?
assistant: I'm doing great, thanks for asking!</example>
<example>user: Where's the closest shop for dill?
assistant: Let's use our store lookup tool to find the closest shop for dill.</example>
<example>user: Can you place an order for 1lbs of salmon and 2lbs of potatoes?
assistant: Let's invoke our ordering tool and get that done.</example>
<example>user: I need to find a substitute for honey.
assistant: Let's use our substitute lookup tool to find a substitute for honey.</example>
<example>user: I need a recipe for mashed potatoes.
assistant: Let's use our recipe lookup tool to find a recipe for mashed potatoes.</example>
</examples>

In [4]:
"""
Multi-shot provider native: We provide examples of user inputs and assistant tool calls in our system prompt.
We use provider-native Assistant message objects, so that the tool calls include provider-specific parameters.
"""

from src.examples.multishot_provider_native import (
    init_agent as init_multishot_native_agent,
)


async def multishot_native():
    agent = init_multishot_native_agent()
    await agent(shop_query)
    print_conversation(agent)


await multishot_native()

system: You are an experienced chef with a passion for creating delicious homemade meals.
You'll be working with your teammate to prepare a meal for Thanksgiving.

You have a set of tools at your disposal, which you may invoke when appropriate.

Here are some examples to help you decide when to use which tool:
<examples>
<example>[{'role': 'user', 'content': 'Hey, how are you?'}, {'role': 'assistant', 'content': "I'm doing great, thanks for asking!"}]</example>
<example>[{'role': 'user', 'content': "Where's the closest shop for dill?"}, {'role': 'assistant', 'content': "Let's use our store lookup tool to find the closest shop for dill.", 'tool_calls': [{'id': '2372fee1-1f62-47d6-835d-c0bc29f2236e', 'type': 'function', 'function': {'name': 'find_nearby_stores', 'arguments': '{"ingredient": "dill", "location": "Financial District"}'}}]}]</example>
<example>[{'role': 'user', 'content': 'Can you place an order for 1lbs of salmon and 2lbs of potatoes?'}, {'role': 'assistant', 'content': "Le